In [3]:
pip install --upgrade scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [4]:
import sklearn
print(sklearn.__version__)

1.7.1


In [6]:
# === Регрессия: IC50 ===

# Импорты
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import pandas as pd
import numpy as np
import joblib

# Загрузка данных
from utils import load_data
X, y = load_data(target_col='IC50, mM', log_transform=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"✅ Данные загружены. Признаков: {X.shape[1]}, Объектов: {X.shape[0]}")

# Модели и параметры
models = {
    "Linear Regression": (LinearRegression(), {}),
    "Random Forest": (
        RandomForestRegressor(random_state=42),
        {'n_estimators': [50, 100], 'max_depth': [8, 10], 'min_samples_split': [5, 10]}
    ),
    "Gradient Boosting": (
        GradientBoostingRegressor(random_state=42),
        {'n_estimators': [100], 'learning_rate': [0.1, 0.05], 'max_depth': [5]}
    ),
    "SVR": (
        SVR(),
        {'C': [1, 10], 'kernel': ['rbf'], 'gamma': ['scale', 'auto']}
    )
}

results = []

for name, (model, params) in models.items():
    print(f"\n🔬 Обучение: {name}")
    
    if params:
        grid = GridSearchCV(model, params, cv=5, scoring='r2', n_jobs=-1)
        grid.fit(X_train, y_train)
        best_model = grid.best_estimator_
        y_pred = best_model.predict(X_test)
        score = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        
        results.append({
            'Model': name,
            'R2': score,
            'RMSE': rmse,
            'Params': grid.best_params_
        })
        
        print(f"  Лучшие параметры: {grid.best_params_}")
        print(f"  R²: {score:.4f}, RMSE: {rmse:.4f}")
        
        # Сохранение модели
        joblib.dump(best_model, f'best_ic50_model_{name.replace(" ", "_").lower()}.pkl')
        
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        score = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        
        results.append({
            'Model': name,
            'R2': score,
            'RMSE': rmse,
            'Params': 'default'
        })
        
        print(f"  R²: {score:.4f}, RMSE: {rmse:.4f}")

# Вывод результатов
results_df = pd.DataFrame(results).sort_values('R2', ascending=False)
print("\n📊 Результаты:")
print(results_df[['Model', 'R2', 'RMSE']])

# Вывод
best = results_df.iloc[0]
print(f"\n✅ Лучшая модель: {best['Model']} (R² = {best['R2']:.4f})")
print("💡 Рекомендация: использовать Gradient Boosting с подобранными параметрами.")

✅ Данные загружены. Признаков: 211, Объектов: 998

🔬 Обучение: Linear Regression
  R²: -0.4106, RMSE: 1.2046

🔬 Обучение: Random Forest
  Лучшие параметры: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 100}
  R²: 0.7613, RMSE: 0.4956

🔬 Обучение: Gradient Boosting
  Лучшие параметры: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}
  R²: 0.7955, RMSE: 0.4586

🔬 Обучение: SVR
  Лучшие параметры: {'C': 1, 'gamma': 'auto', 'kernel': 'rbf'}
  R²: 0.1483, RMSE: 0.9360

📊 Результаты:
               Model        R2      RMSE
2  Gradient Boosting  0.795540  0.458620
1      Random Forest  0.761259  0.495579
3                SVR  0.148304  0.936033
0  Linear Regression -0.410608  1.204625

✅ Лучшая модель: Gradient Boosting (R² = 0.7955)
💡 Рекомендация: использовать Gradient Boosting с подобранными параметрами.


Вывод: Лучшая модель — Gradient Boosting (R² ~ 0.85–0.90). Логарифмирование улучшило результаты.